In [ ]:
# MINI BATCH GRADIENT DESCENT

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

df = pd.read_csv(r"C:\Users\Krishnasai\kc_house_data.csv")
df

df.info()

df.describe()

df.corr()

# Missing values

df.isnull().sum()

#There are no null values

# Outliers Treatment

data_x = df['sqft_living']
data_y = df['price']

plt.scatter(data_x,data_y)
plt.xlabel("house sqt feet")
plt.ylabel("price of house")

sns.boxplot(x=df['sqft_living'])

sns.boxplot(x=df['price'])

#we are having few outliners in the data set,
#we need to remove them

df = df[['sqft_living','price']]
z = np.abs(stats.zscore(df))
filter = df[(z < 3).all(axis=1)]
print(filter)
print("\nlength of data:",len(df))
print("lenth of filter:",len(filter))

sns.boxplot(x=filter['sqft_living'])

sns.boxplot(x=filter['price'])

df

plt.scatter(filter['sqft_living'],filter['price'])

q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
iqr

print((df < (q1 - iqr*1.5)) | (df > (q3 + iqr*1.5)))

df.shape

data_cleared = df[~((df < (q1 - 1.5 * iqr)) |(df > (q3 + 1.5 * iqr))).any(axis=1)]
data_cleared.shape

sns.boxplot(x=data_cleared['sqft_living'])

sns.boxplot(x=data_cleared['price'])

plt.scatter(data_x,data_y)


plt.scatter(data_cleared['sqft_living'],data_cleared['price'])

data_cleared.shape

data_x = data_cleared['sqft_living']
data_y = data_cleared['price']

# Normalisation of data set

data_x = (data_x - data_x.min())/(data_x.max() - data_x.min())
data_y = (data_y - data_y.min())/(data_y.max() - data_y.min())
data_x = np.array(data_x)
data_y = np.array(data_y)

data2 = data_cleared.copy()

data_new = []
for i in range(len(data_x)):
    data_new.append([data_x[i],data_y[i]])
data_new[:5]

data_new = pd.DataFrame(data_new,columns=['sqft_living','price'])
data_new

# Splitting data into Training and Testing data

from sklearn.model_selection import train_test_split

data_train,data_test = train_test_split(data_new,test_size=0.1)

# INITIALISING BATCH SIZE AND CALCULATING NUMBER OF BATCHES
bs = 1000
nb = np.ceil(len(data_train)/bs)
nb

# Training the Model

# INITIALISING ALL REQUIRED PARAMETERS
iter = 1
# epochs = 10
epochs = 100
batch = nb - 1
bat = 1
alpha = 0.1 # LEARNING RATE
m = 1
c = -1
error = []

data_train

data_x = np.array(data_train['sqft_living'])
data_y = np.array(data_train['price'])
len(data_x)

count = 0
while (iter<=epochs):
  i = 0
  # print("iter =",iter)
  bat = 1
  # RUNNING ITERATION FOR N-1 BATCHES
  while (bat <= batch):
    der_m = 0
    der_c = 0
    err = 0
    # print("bat = ",bat)
    for k in range(0,bs):
      der_m = der_m + (data_y[i] - (m*data_x[i])-c)*(data_x[i])
      der_c = der_c + (data_y[i] - (m*data_x[i])-c)
      i = i+1
      
    err = der_c**2
    err = (1/(2*bs))*err
    error.append(err)
    
    
    der_m = (-1/bs)*der_m
    der_c = (-1/bs)*der_c
    change_m = (-1)*alpha*der_m
    change_c = (-1)*alpha*der_c
    
    m = m + change_m
    c = c + change_c
    bat = bat + 1
    # print("new bat = ",bat,"\n")
    # EXECUTING FOR LAST BATCH
  der_m = 0
  der_c = 0
  err = 0
  count = count + 1
  # print("end")
  
  # FINDING REMAINING NUMBER OF BATCHES
  es = len(data_x) - i  
  
  for k in range(i,len(data_x)):
    der_m = der_m + (data_y[i] - (m*data_x[i])-c)*(data_x[i])
    der_c = der_c + (data_y[i] - (m*data_x[i])-c)
    i = i + 1
  
  err = der_c**2
  err = (1/(2*es))*err
  error.append(err)

  der_m = (-1/es)*der_m
  der_c = (-1/es)*der_c
  
  change_m = (-1)*alpha*der_m
  change_c = (-1)*alpha*der_c
  
  m = m + change_m
  c = c + change_c
  
  iter = iter + 1
  # print("new iter",iter,"\n")

print(m,c)

print(count)

# MSE of training data

# MEAN SQUARE ERROR IN EXPONENTIAL FORM
# MINIMISED ERROR
mse = 0
for i in range(0,len(data_train)):
    mse += (data_y[i] - (m*data_x[i]) -c)**2
mse = (1/len(data_train))*mse
mse

data_test = np.array(data_test)
data_test_x = data_test[:,0]
print(data_test_x)
data_test_y = data_test[:,1]
data_test_y

len(data_test)


# predicting results of test data set
pred = []
for i in range(len(data_test_x)):
  # print((m*data_test_x[i]) +c)
  pred.append((m*data_test_x[i]) +c)

# pred

# MSE of testing data

mse = 0
for i in range(0,len(data_test)):
  mse += (data_test[i][1] - (m*data_test[i][0]) - c)**2
mse = mse/len(data_test)
mse

# Visualising testing data results

plt.scatter(data_test_x,data_test_y,label="Original Data points",color='orange')
plt.plot(data_test_x,pred,label="predicted regression line")
plt.title("House Price Prediction using Mini Batch Gradient Descent")
plt.xlabel("Data x")
plt.ylabel("Data y")
plt.grid()
plt.legend()
plt.show()

data_train = pd.DataFrame(data_train)

data_train_pred = m*data_train['sqft_living'] + c
data_train_pred

# Visualising results of Training data¶

plt.plot(data_train['sqft_living'],data_train_pred,'red',label="Regression Line")
plt.scatter(data_train['sqft_living'],data_train['price'],label="original data points")
plt.xlabel("square feet of living area")
plt.ylabel("price of house")
plt.legend()
plt.grid()
plt.title("Training Results Analysis")
plt.show()
%matplotlib inline

# ERROR RATE ANALYSIS

iters = np.arange(epochs*nb)
plt.plot(iters,error,color='m',label="error rate at ith iteration")
plt.legend()
plt.xlabel("Number of iteration")
plt.ylabel("Error Rate at particular Iteration")
plt.title("Error Analysis")
plt.grid()
plt.show()

len(error)

# computig mean squared error
mse = 0
mse = (data_train['price'] - m*data_train['sqft_living'] - c)**2
mse.sum()/len(data_train)

min_x = data2['sqft_living'].min()
max_x = data2['sqft_living'].max()
min_y = data2['price'].min()
max_y = data2['price'].max()
print(min_x,min_y,max_x,max_y)

# Deployment of model

input_x = float(input("enter square feet"))
input_x = (input_x - min_x)/(max_x - min_x)
output_y = m*input_x + c
output_y = (output_y)*(max_y - min_y) + min_y
output_y



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  